In [2]:
import argparse
import os
from functools import partial

import numpy as np
import paddle
from data import convert_pairwise_example
from data import read_text_pair
from model import PairwiseMatching
import copy

from paddlenlp.data import Pad, Tuple
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import AutoModel, AutoTokenizer

/home/siyuan/miniconda3/envs/search/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/siyuan/miniconda3/envs/search/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [3]:
paddle.set_device("cpu")

Place(cpu)

In [4]:
@paddle.no_grad()
def predict(model, data_loader): 
    model.eval()

    batch_probs = [] 
    for batch_data in data_loader:
        input_ids, token_type_ids = batch_data
        batch_prob = model.predict(input_ids=input_ids, token_type_ids=token_type_ids).numpy() 
        batch_probs.append(batch_prob)

    conca_batch_probs = np.concatenate(batch_probs, axis=0) 

    return conca_batch_probs

In [5]:
# 下面开始搭建模型，并加载参数

In [6]:
model_name = "ernie-3.0-medium-zh"

pretrained_model = AutoModel.from_pretrained(model_name)

[2024-01-28 18:45:48,480] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieModel'> to load 'ernie-3.0-medium-zh'.
[2024-01-28 18:45:48,481] [    INFO] - Already cached /home/siyuan/.paddlenlp/models/ernie-3.0-medium-zh/model_state.pdparams
[2024-01-28 18:45:48,482] [    INFO] - Loading weights file model_state.pdparams from cache at /home/siyuan/.paddlenlp/models/ernie-3.0-medium-zh/model_state.pdparams
[2024-01-28 18:45:48,773] [    INFO] - Loaded weights file from disk, setting weights to model.
[2024-01-28 18:45:52,914] [ WARNING] - Some weights of the model checkpoint at ernie-3.0-medium-zh were not used when initializing ErnieModel: ['ernie.encoder.layers.6.linear1.bias', 'ernie.encoder.layers.6.linear1.weight', 'ernie.encoder.layers.6.linear2.bias', 'ernie.encoder.layers.6.linear2.weight', 'ernie.encoder.layers.6.norm1.bias', 'ernie.encoder.layers.6.norm1.weight', 'ernie.encoder.layers.6.norm2.bias', 'ernie.encoder.layers.6.norm2.weight', 'ernie.encode

In [7]:
model = PairwiseMatching(pretrained_model)

params_path = "model_param/model_400/model_state.pdparams" 

if params_path and os.path.isfile(params_path): 
    state_dict = paddle.load(params_path)
    model.set_dict(state_dict)
    print("Loaded parameters from %s" % params_path)
else:
    raise ValueError("Please set --params_path with correct pretrained model file")

Loaded parameters from model_param/model_400/model_state.pdparams


In [8]:
# 下面开始构造测试数据的加载器

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2024-01-28 18:45:53,291] [    INFO] - We are using (<class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'>, False) to load 'ernie-3.0-medium-zh'.
[2024-01-28 18:45:53,293] [    INFO] - Already cached /home/siyuan/.paddlenlp/models/ernie-3.0-medium-zh/ernie_3.0_medium_zh_vocab.txt
[2024-01-28 18:45:53,313] [    INFO] - tokenizer config file saved in /home/siyuan/.paddlenlp/models/ernie-3.0-medium-zh/tokenizer_config.json
[2024-01-28 18:45:53,313] [    INFO] - Special tokens file saved in /home/siyuan/.paddlenlp/models/ernie-3.0-medium-zh/special_tokens_map.json


In [10]:
trans_func = partial(convert_pairwise_example, tokenizer=tokenizer, max_seq_length=64, phase="predict")

In [11]:
def batchify_fn(samples):
    fn = Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id, dtype="int64"),  
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id, dtype="int64"),  
    )

    processed_samples = fn(samples) 

    result = []
    for data in processed_samples:
        result.append(data) 

    return result

In [18]:
test_data_path = "rank_dataset/test_pairwise.csv" 

test_ds = load_dataset(read_text_pair, data_path=test_data_path, lazy=False)

In [19]:
print(test_ds[0])

{'query': '中西方语言与文化的差异', 'title': '中西方文化差异以及语言体现中西方文化,差异,语言体现'}


In [20]:
test_ds_copy = copy.deepcopy(test_ds) 

batch_sampler = paddle.io.BatchSampler(test_ds, batch_size=128, shuffle=False)

test_data_loader = paddle.io.DataLoader(dataset=test_ds.map(trans_func), batch_sampler=batch_sampler, collate_fn=batchify_fn, return_list=True)

In [21]:
print(test_ds[0])
print(test_ds_copy[0])

([1, 12, 213, 58, 405, 545, 54, 68, 73, 5, 859, 712, 2, 12, 213, 58, 68, 73, 859, 712, 22, 107, 405, 545, 82, 87, 12, 213, 58, 68, 73, 30, 859, 712, 30, 405, 545, 82, 87, 2], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
{'query': '中西方语言与文化的差异', 'title': '中西方文化差异以及语言体现中西方文化,差异,语言体现'}


In [ ]:
# 下面开始计算相似度

In [22]:
y_probs = predict(model, test_data_loader)

/home/siyuan/miniconda3/envs/search/lib/python3.10/site-packages/paddlenlp/transformers/tokenizer_utils_base.py:3049: UserWarning: Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
  warnings.warn(


In [23]:
rank_result = [] 

for idx, prob in enumerate(y_probs): 
    text_pair = test_ds_copy[idx] 
    text_pair["pred_prob"] = prob[0] 
    rank_result.append(text_pair) 
    print(text_pair) 

{'query': '中西方语言与文化的差异', 'title': '中西方文化差异以及语言体现中西方文化,差异,语言体现', 'pred_prob': 0.9722315}
{'query': '中西方语言与文化的差异', 'title': '论中西方文化差异在非言语交际中的体现中西方文化,差异,非言语交际', 'pred_prob': 0.91743886}
{'query': '中西方语言与文化的差异', 'title': '中西方体态语文化差异跨文化,体态语,非语言交际,差异', 'pred_prob': 0.9152562}
{'query': '中西方语言与文化的差异', 'title': '由此便可以发现两种语言以及两种文化的差异。', 'pred_prob': 0.86496}
{'query': '中西方语言与文化的差异', 'title': '文化空缺视域下的中西方体态语对比研究体态语;中西方差异;文化空缺;跨文化交际', 'pred_prob': 0.8663044}
{'query': '中西方语言与文化的差异', 'title': '浅析中西方文化差异在语言中的体现及其对翻译的影响中西方文化,差异,语言,翻译,影响', 'pred_prob': 0.9291804}
{'query': '中西方语言与文化的差异', 'title': '跨文化交流中文化差异对不同语言运用的影响跨文化交流,语言运用,价值取向,审美观', 'pred_prob': 0.8729114}
{'query': '中西方语言与文化的差异', 'title': '文化与语言的关系在中西文化中的映射交际,符号,语言,文化', 'pred_prob': 0.9024463}
{'query': '中西方语言与文化的差异', 'title': '从中西方文化价值差异看跨文化交际——以电影《推手》为例中西方文化,差异,跨文化交际', 'pred_prob': 0.8748074}
{'query': '中西方语言与文化的差异', 'title': '跨文化交际中的文化误读研究文化误读,影响,中华文化,西方文明', 'pred_prob': 0.79062104}
{'query': '中西方语言与文化的差异', 'title': '中西方文化差异在翻译中的体现中西方,文化差异,

In [24]:
rank_result.sort(key=lambda x: x['pred_prob'], reverse=True) # 把rank_result中的各元素，按照'pred_prob'键对应的值从大到小排序

for i in rank_result:
    print(i)

{'query': '中西方语言与文化的差异', 'title': '中西方文化差异以及语言体现中西方文化,差异,语言体现', 'pred_prob': 0.9722315}
{'query': '中西方语言与文化的差异', 'title': '中西方文化差异研究中西方文化,文化差异,研究', 'pred_prob': 0.9567481}
{'query': '中西方语言与文化的差异', 'title': '中西方文化差异的几个显著现象及其原因分析中西方文化,文化差异,词汇,亲属关系,法律', 'pred_prob': 0.94793624}
{'query': '中西方语言与文化的差异', 'title': '谈中西方文化差异东西方文化,差异', 'pred_prob': 0.94497424}
{'query': '中西方语言与文化的差异', 'title': '中西方问候语的文化差异问候语,文化差异,文化背景', 'pred_prob': 0.93141174}
{'query': '中西方语言与文化的差异', 'title': '论如何应对中西方文化差异中西方,语言,价值观,社会,应对', 'pred_prob': 0.93046385}
{'query': '中西方语言与文化的差异', 'title': '浅析中西方文化差异在语言中的体现及其对翻译的影响中西方文化,差异,语言,翻译,影响', 'pred_prob': 0.9291804}
{'query': '中西方语言与文化的差异', 'title': '中西方文化差异与翻译中西方,文化差异,翻译影响,策略方法,译者素质', 'pred_prob': 0.92791545}
{'query': '中西方语言与文化的差异', 'title': '浅谈翻译中的中西方文化差异-以语言文化差异为例翻译,中西方文化,语言,差异', 'pred_prob': 0.9277191}
{'query': '中西方语言与文化的差异', 'title': '试论中西方语言文化教育的差异比较与融合中西方,语言文化教育,差异', 'pred_prob': 0.92671293}
{'query': '中西方语言与文化的差异', 'title': '中西方文化差异对跨文化交际的影响跨文化交际,文化差异,语言,观念,思维', '